# Temperatura Global

![global_map](images/global_map.png)


**Fuentes:** 
+ https://data.giss.nasa.gov/gistemp/
+ https://www.metoffice.gov.uk/hadobs/
+ https://www.ncdc.noaa.gov/cag/global/time-series

[Descarga directa de los datos de anomalías de temperatura (global)](https://www.metoffice.gov.uk/hadobs/hadcrut4/data/current/time_series/HadCRUT.4.6.0.0.monthly_ns_avg.txt)

[Descarga directa de los datos de anomalías de temperatura 2 (global)](https://www.metoffice.gov.uk/hadobs/hadcrut5/)

[Descarga directa de los datos de anomalías de temperatura (mapa)](https://data.giss.nasa.gov/pub/gistemp/gistemp1200_GHCNv4_ERSSTv5.nc.gz)

In [ ]:
# librerias

import numpy as np

import pandas as pd  # dataframe
import xarray as xr

import pylab as plt   # graficos
from matplotlib.animation import FuncAnimation


import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature  # mapa global


%matplotlib notebook
plt.rcParams['animation.embed_limit'] = 2**128

# prediccion
from fbprophet import Prophet

import warnings
warnings.filterwarnings('ignore')

## Anomalías de temperatura por meses

In [ ]:
# caragar datos

data=pd.read_csv('data/HadCRUT.5.0.1.0.analysis.summary_series.global.monthly.csv')

data.head()

In [ ]:
# transformar datos

data['year']=data.Year.apply(lambda x: int(str(x).split('.')[0]))
data['month']=data.Year.apply(lambda x: int(str(x).split('.')[1]))

data.drop('Year', axis=1, inplace=True)

data.rename(columns={'Anomaly': 'anomaly'}, inplace=True)

months=dict(zip(data.month.unique(), [i for i in range(1, 13)]))

data.month=data.month.apply(lambda x: months[x])

data.head()

In [ ]:
data.year.min(), data.year.max()

In [ ]:
months=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [ ]:
plt.plot(months, data.anomaly[data.year==1880]);

In [ ]:
fig=plt.figure(figsize=(8,5))
ax=plt.subplot(111)

plt.ylim(-4, 4)


def animate_func(i):
 
    plt.title(f'Año {1880 + i}')

    data_plot=data.anomaly[data.year==1880 + i]    
    
    ax.plot(months[:len(data_plot)], 
            data_plot, 
            c=plt.cm.viridis(i*2));
    
    return ax



anim=FuncAnimation(fig, 
                   animate_func, 
                   frames=len(data.year.unique()),
                   interval=200, 
                   repeat=False)

### Anomalías de temperatura por meses (visualización polar, en espiral)

In [ ]:
data_polar=pd.read_csv('data/HadCRUT.5.0.1.0.analysis.summary_series.global.monthly.polar.csv')

data_polar.head()

In [ ]:
data_polar['year']=data_polar.time.apply(lambda x: x.split('-')[0]).astype(int)
data_polar['month']=data_polar.time.apply(lambda x: x.split('-')[1]).astype(int)

data_polar.head()

In [ ]:
data_polar.year.max()

In [ ]:
# creacion de array de angulos
theta=np.linspace(0, 2*np.pi, 12)  # para 12 meses, representacion de datos

# inicia figura
fig=plt.figure(figsize=(9,9))
ax=plt.subplot(111, projection='polar')


# circulos informativos, azul=0ºC, rojo=1.5 y 3ºC por encima de la media
ax.plot(np.linspace(0, 2*np.pi, 1000), [1. for _ in range(1000)], c='blue')
ax.plot(np.linspace(0, 2*np.pi, 1000), [2.5 for _ in range(1000)], c='red')
ax.plot(np.linspace(0, 2*np.pi, 1000), [3. for _ in range(1000)], c='red')

ax.text(np.pi/2, 1.0, '0.0 C', color='blue', ha='center', fontdict={'fontsize': 20})
ax.text(np.pi/2, 2.5, '1.5 C', color='red', ha='center', fontdict={'fontsize': 20})
ax.text(np.pi/2, 3.0, '2.0 C', color='red', ha='center', fontdict={'fontsize': 20})

# elimina etiquetas
ax.axes.get_yaxis().set_ticklabels([])
ax.axes.get_xaxis().set_ticklabels([])

# setup
ax.set_ylim(0, 3.25)
ax.set_facecolor('#000100')
ax.grid(False)

# bucle para representar cada año
years=data_polar.year.unique()


for i, year in enumerate(years):
    
    # datos
    data_plot=data_polar[data_polar.year==year].anomaly 
    
    # visualizacion
    ax.set_title('Anomalía Temperatura Global (1850-2021)', 
                  color='black', 
                  fontdict={'fontsize': 20})

    ax.plot(theta[:len(data_plot)], 
            data_plot, 
            c=plt.get_cmap('jet')(i));

In [ ]:
# creacion de array de angulos
theta=np.linspace(0, 2*np.pi, 12)  # para 12 meses, representacion de datos

# inicia figura
fig=plt.figure(figsize=(9,9))
ax=plt.subplot(111, projection='polar')


# circulos informativos, azul=0ºC, rojo=1.5 y 3ºC por encima de la media
ax.plot(np.linspace(0, 2*np.pi, 1000), [1. for _ in range(1000)], c='blue')
ax.plot(np.linspace(0, 2*np.pi, 1000), [2.5 for _ in range(1000)], c='red')
ax.plot(np.linspace(0, 2*np.pi, 1000), [3. for _ in range(1000)], c='red')

ax.text(np.pi/2, 1.0, '0.0 C', color='blue', ha='center', fontdict={'fontsize': 20})
ax.text(np.pi/2, 2.5, '1.5 C', color='red', ha='center', fontdict={'fontsize': 20})
ax.text(np.pi/2, 3.0, '2.0 C', color='red', ha='center', fontdict={'fontsize': 20})

# elimina etiquetas
ax.axes.get_yaxis().set_ticklabels([])
ax.axes.get_xaxis().set_ticklabels([])

# setup
ax.set_ylim(0, 3.25)
ax.set_facecolor('#000100')
ax.grid(False)


# bucle para representar cada año
years=data_polar.year.unique()


months_angles=np.linspace((np.pi/2)+(2*np.pi), np.pi/2, 13)

for i,month in enumerate(months):
    ax.text(months_angles[i], 3.7 ,month, color='black', fontsize=15, ha='center')

def animate_func(i):
    # Borra el ultimo año del centro y deja el ultimo
    for txt in ax.texts:
      if(txt.get_position()==(0,0)) and i!=len(years):
        txt.set_visible(False)
        
    # datos
    data_plot=data_polar[data_polar.year==years[i]].anomaly
    
    # visualizacion
    ax.plot(theta[:len(data_plot)], 
            data_plot, 
            c=plt.get_cmap('jet')(i))
    
    
    ax.text(0 , 0, years[i], fontsize=20, color='white',ha='center')
    
    return ax



anim=FuncAnimation(fig, 
                   animate_func, 
                   frames=len(years), 
                   interval=100,
                   repeat=False)

### Predicción de anomalías hasta 2050

In [ ]:
data_polar.info()

In [ ]:
df_prophet=data_polar[['time', 'anomaly']]

df_prophet.time=pd.to_datetime(df_prophet.time)

df_prophet.columns=['ds', 'y']

df_prophet.info()

In [ ]:
%%time
model=Prophet().fit(df_prophet)

In [ ]:
# desde marzo del 2022 hasta diciembre del 2050

future=model.make_future_dataframe(periods=347, freq='M')

forecast=model.predict(future)

fig=model.plot(forecast)

In [ ]:
forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']][forecast.ds>'2022-03-01']

forecast.columns=['time', 'anomaly', 'lower', 'upper']

forecast.head()

In [ ]:
# añadimos algo de desviacion a la prediccion, para el plot (comentar esta casilla para ver prediccion en crudo)

forecast_noise=pd.DataFrame()

forecast_noise['time']=forecast.time

forecast_noise['anomaly']=np.random.uniform(forecast.lower, forecast.upper)

forecast_noise.head()

In [ ]:
data_polar[['time', 'anomaly']].tail()

In [ ]:
total_df=pd.concat([data_polar[['time', 'anomaly']], forecast_noise], axis=0)

total_df.time=total_df.time.astype(str)

total_df['year']=total_df.time.apply(lambda x: x.split('-')[0]).astype(int)
total_df['month']=total_df.time.apply(lambda x: x.split('-')[1]).astype(int)

total_df.head()

In [ ]:
total_df.tail()

In [ ]:
# creacion de array de angulos
theta=np.linspace(0, 2*np.pi, 12)  # para 12 meses, representacion de datos

# inicia figura
fig=plt.figure(figsize=(9,9))
ax=plt.subplot(111, projection='polar')


# circulos informativos, azul=0ºC, rojo=1.5 y 3ºC por encima de la media
ax.plot(np.linspace(0, 2*np.pi, 1000), [1. for _ in range(1000)], c='blue')
ax.plot(np.linspace(0, 2*np.pi, 1000), [2.5 for _ in range(1000)], c='red')
ax.plot(np.linspace(0, 2*np.pi, 1000), [3. for _ in range(1000)], c='red')

ax.text(np.pi/2, 1.0, '0.0 C', color='blue', ha='center', fontdict={'fontsize': 20})
ax.text(np.pi/2, 2.5, '1.5 C', color='red', ha='center', fontdict={'fontsize': 20})
ax.text(np.pi/2, 3.0, '2.0 C', color='red', ha='center', fontdict={'fontsize': 20})

# elimina etiquetas
ax.axes.get_yaxis().set_ticklabels([])
ax.axes.get_xaxis().set_ticklabels([])

# setup
ax.set_ylim(0, 3.25)
ax.set_facecolor('#000100')
ax.grid(False)


# bucle para representar cada año
years=total_df.year.unique()


months_angles=np.linspace((np.pi/2)+(2*np.pi), np.pi/2, 13)

for i,month in enumerate(months):
    ax.text(months_angles[i], 3.7 ,month, color='black', fontsize=15, ha='center')

def animate_func(i):
    # Borra el ultimo año del centro y deja el ultimo
    for txt in ax.texts:
      if(txt.get_position()==(0,0)) and i!=len(years):
        txt.set_visible(False)
        
    # datos
    data_plot=total_df[total_df.year==years[i]].anomaly
    
    # visualizacion
    ax.plot(theta[:len(data_plot)], 
            data_plot, 
            c=plt.get_cmap('jet')(i))
    
    
    ax.text(0 , 0, years[i], fontsize=20, color='white',ha='center')
    
    return ax



anim=FuncAnimation(fig, 
                   animate_func, 
                   frames=len(years), 
                   interval=100,
                   repeat=False)

## Mapas

In [ ]:
map_df=xr.open_dataset('data/gistemp1200_GHCNv4_ERSSTv5.nc')

map_df

In [ ]:
# anual

climate=map_df.resample(time='Y').mean()

anomaly=climate['tempanomaly']

#anomaly

In [ ]:
# se crea un mapamundi

fig = plt.figure(figsize=(10,8))
fig.subplots_adjust(left=0.02, bottom=0.04, right=0.98, top=0.96, wspace=0, hspace=0)

ax=fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())

ax.add_feature(NaturalEarthFeature('cultural', 'admin_0_countries', '10m'),
                       facecolor='none', edgecolor='black')

plt.show();

In [ ]:
# se crea mapa para un año dado

# configuracion de la barra de color
cbar_kwargs = {
    'orientation':'horizontal',
    'fraction': 0.045,
    'pad': 0.01,
    'extend':'neither'
}

fig=plt.figure(figsize=(10,8))
fig.subplots_adjust(left=0.02, bottom=0.04, right=0.98, top=0.96, wspace=0, hspace=0)

ax=fig.add_subplot(1,1,1, projection = ccrs.PlateCarree())

ax.add_feature(NaturalEarthFeature('cultural', 'admin_0_countries', '10m'),
                       facecolor='none', edgecolor='black')

ax.set_extent([-150, 150, -55, 85])

i=-1

date=pd.to_datetime(anomaly.isel(time=i)['time'].values)

ax.set_title('Anomalía de Temperatura en '+ str(date.year) + ' [°C]')

anomaly.isel(time=i).plot.imshow(ax=ax, add_labels=False, add_colorbar=True,
               vmin=-4, vmax=4, cmap='coolwarm',
                                 
               cbar_kwargs=cbar_kwargs, interpolation='bicubic')

plt.show();

In [ ]:
fig=plt.figure(figsize=(10,8))
fig.subplots_adjust(left=0.02, bottom=0.04, right=0.98, top=0.96, wspace=0, hspace=0)


ax = fig.add_subplot(111, projection=ccrs.PlateCarree())


ax.add_feature(NaturalEarthFeature('cultural', 'admin_0_countries', '10m'),
              facecolor='none', edgecolor='black')

ax.set_extent([-110, 110, -55, 85])

image=anomaly.isel(time=0).plot.imshow(ax=ax, 
                                       vmin=-4, 
                                       vmax=4,
                                       add_labels=False,
                                       cmap='coolwarm', 
                                       animated=True,
                                       cbar_kwargs=cbar_kwargs, 
                                       interpolation='bicubic')



def animate(t):
    
    date=pd.to_datetime(anomaly.sel(time=t)['time'].values)
    
    ax.set_title("Anomalía de Temperatura en " + str(date.year) + " [°C]")
    ax.title.set_fontsize(18)
    
    image.set_array(anomaly.sel(time=t))
    
    return image

ani=FuncAnimation(fig, 
                  animate, 
                  frames=anomaly['time'].values,
                  interval=100,
                  repeat=False)
